# Unified RNN Model Evaluation Notebook

This notebook provides comprehensive evaluation of the unified RNN model for the BARYONYX Lost & Found system.

## Overview

The unified RNN model combines three tasks in a single, efficient architecture:

1. **User Behavior Prediction**: Predicts next user action based on behavior sequences
2. **Text Description Classification**: Classifies item descriptions into categories (phone, mouse, wallet, tumbler)
3. **Temporal Pattern Recognition**: Predicts optimal time periods for finding items

## Table of Contents

1. [Setup and Imports](#setup)
2. [Load Unified Model](#load)
3. [User Behavior Evaluation](#behavior)
4. [Description Classification Evaluation](#description)
5. [Temporal Pattern Evaluation](#temporal)
6. [Performance Analysis](#analysis)
7. [Visualization and Results](#visualization)
8. [Model Comparison](#comparison)


## 1. Setup and Imports {#setup}


In [ ]:
# Core imports
import os
import sys
import json
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# PyTorch imports
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset

# Machine Learning imports
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    classification_report, confusion_matrix, roc_auc_score
)
from sklearn.model_selection import train_test_split
from scipy.stats import pearsonr, spearmanr

# Add project root to path
sys.path.append('.')

# Import unified model
from unified_rnn_model import UnifiedRNNManager

print("✅ All imports successful!")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name(0)}")

# Set up plotting style
plt.style.use('default')
sns.set_palette("husl")


## 2. Load Unified Model {#load}


In [ ]:
# Initialize device and load unified model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Initialize unified RNN manager
print("Loading Unified RNN Model...")
manager = UnifiedRNNManager(device=str(device))

# Check if model files exist
model_dir = 'models/unified_rnn'
model_files = {
    'unified_rnn_model.pth': 'Unified RNN Model',
    'vocab.pkl': 'Vocabulary',
    'metadata.json': 'Model Metadata'
}

print("\n=== Model File Status ===")
for filename, description in model_files.items():
    filepath = os.path.join(model_dir, filename)
    if os.path.exists(filepath):
        file_size = os.path.getsize(filepath) / 1024  # KB
        print(f"✅ {description}: {filename} ({file_size:.1f} KB)")
    else:
        print(f"❌ {description}: {filename} - NOT FOUND")

# Load the model
success = manager.load_model()
if success:
    print("\n✅ Unified RNN Model loaded successfully!")
else:
    print("\n⚠️ Model loading failed, using initialized model")

# Display model information
print(f"\n=== Model Information ===")
info = manager.get_model_info()
for key, value in info.items():
    print(f"{key}: {value}")

# Test model components
print(f"\n=== Model Component Tests ===")

# Test behavior prediction
sample_behavior = torch.randn(1, 10, 10).to(device)
behavior_pred, behavior_probs, behavior_attn = manager.predict_behavior(sample_behavior)
print(f"Behavior Prediction: {behavior_pred}, Confidence: {behavior_probs.max():.3f}")

# Test temporal prediction
sample_temporal = torch.randn(1, 7, 10).to(device)
temporal_pred, temporal_probs, time_periods = manager.predict_temporal_pattern(sample_temporal)
print(f"Temporal Prediction: {temporal_pred} ({time_periods[temporal_pred]}), Confidence: {temporal_probs.max():.3f}")

# Test description classification (if vocabulary is loaded)
if manager.vocab is not None:
    test_text = "Lost black iPhone 12 with cracked screen"
    desc_pred, desc_probs, desc_attn, categories = manager.classify_description(test_text)
    print(f"Description Classification: '{test_text}' -> {categories[desc_pred]} (conf: {desc_probs.max():.3f})")
else:
    print("Description Classification: Vocabulary not loaded, will build from test data")
